In [265]:
import os
import re
import csv
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [297]:
URL = "https://www.hemnet.se/bostader?expand_locations=100000"

In [267]:
path = 'C:\\Users\\Admin\\Documents\\chromedriver_win32\\chromedriver.exe'
# path = 'write the path to your webdriver'

In [268]:
# initializing the webdriver
def init_driver():
    driver=webdriver.Chrome(path)
    driver.wait = WebDriverWait(driver, 5)
    return driver

In [269]:
def get_Property_Image(soup):
    if soup.find('button', class_ = 'gallery-carousel__image-touchable'):
        return soup.find('button', class_ = 'gallery-carousel__image-touchable').find('img')['src']
    elif soup.find('li', class_ = "slide"):
        return soup.find('li', class_ = "slide").find('button', class_="gallery-carousel__image-touchable").find('img')['src']
    else:
        return None

In [270]:
def get_Contact_name(soup):
    try:
        return soup.find('p', class_="broker-card__text").find('strong').get_text().strip()
    except:
        return None

In [271]:
def get_Contact_Tel(soup):
    try:
        return soup.find('div', class_ =  "qa-broker-phone").find('a', class_ = "broker-card__contact-button--link").get_text().strip()
    except:
        return None

In [272]:
def get_Driftkostnad(soup):
    if soup.find('dt', text='Driftkostnad'):
        return soup.find('dt', text='Driftkostnad').findNext('dd').get_text()
    elif soup.find('dt', text='Pris/m²'):
        return soup.find('dt', text='Pris/m²').findNext('dd').get_text()
    else:
        return None

In [273]:
def get_TotalArea(soup):
    if soup.find('td', text='Tomtarea'):
        return soup.find('td', text='Tomtarea').findNext('dd').get_text()
    elif soup.find('dt', class_ = "property-attributes-table__label", text='Boarea'):
        return  soup.find('dt', class_ = "property-attributes-table__label", text='Boarea').findNext('dd').get_text()
    else:
        return None

In [274]:
def get_Description(soup):
    if soup.find('div',  class_ = "property-description js-property-description property-description--long"):
        return soup.find('div',  class_ = "property-description js-property-description property-description--long").find('p').get_text()
    elif soup.find('div',  class_ = "property-description js-property-description property-description--expanded"):
        return soup.find('div',  class_ = "property-description js-property-description property-description--expanded").find('p').get_text()
    else:
        return None

In [275]:
def get_Price(soup):
    try:
        return soup.find('p', class_ = "property-info__price qa-property-price").text #replace('xc2',' ')
    except:
        return None

In [276]:
def get_Type(soup):
    try:
        return soup.find(string='Bostadstyp').findNext('dd').get_text()
    except:
        return None

In [277]:
def get_Property_Address1(soup):
    try:
        return soup.find('h1', class_ = "qa-property-heading").get_text()
    except:
        return None

In [278]:
def get_Rooms(soup):
    try:
        return  soup.find(string='Antal rum').findNext('dd').get_text()
    except:
        return None

In [279]:
def get_Location(soup):
    try:
        return soup.find('div', class_ = "property-address__area-container").find('span', class_ = "property-address__area").get_text()
    except:
        return None

In [286]:
fieldnames = ['Contact_name','Contact_Tel','Driftkostnad','TotalArea','Description','Price','Type','Property_Address1','Rooms','Location','Property_Image','Property_URL']
df = pd.DataFrame(columns = fieldnames)
df.to_csv('filename.csv')

In [294]:
number = 0

In [295]:
def get_urls(driver,url):
    driver.get(url)
    while(True):
        
        oldlist = driver.find_elements_by_xpath("//li[@class='normal-results__hit js-normal-list-item']/a")  
        for i in range(0, len(oldlist)):
            links = driver.find_elements_by_xpath("//li[@class='normal-results__hit js-normal-list-item']/a")
            driver.execute_script("""
            var div = document.createElement('div');
            div.innerHTML = arguments[0].trim();
            var link = div.childNodes[0];
            link.setAttribute('target','');
            link.click();""", links[i].get_attribute('outerHTML'))
            
            soup=BS(driver.page_source, 'lxml')
            
            lists = []
            
            global number
            num = number+1
            number +=1

            lists.append(num)
            
            lists.append(get_Contact_name(soup))


            lists.append(get_Contact_Tel(soup))


            lists.append(get_Driftkostnad(soup))


            lists.append(get_TotalArea(soup))


            lists.append(get_Description(soup))
            

            lists.append(get_Price(soup))


            lists.append(get_Type(soup))


            lists.append(get_Property_Address1(soup))


            lists.append(get_Rooms(soup))


            lists.append(get_Location(soup))


            try:
                lists.append(driver.find_elements_by_xpath("//button[@class='gallery-carousel__image-touchable']/img")[0].get_attribute('src'))
            except:
                lists.append(get_Property_Image(soup))

            lists.append(driver.current_url)

            with open('filename.csv', "a", newline='') as fp:
                wr = csv.writer(fp, dialect='excel')
                wr.writerow(lists)
                fp.close()
            driver.back()
#           breakdriver.execute_script("window.history.go(-1)")
#           driver.execute_script("""var a=document.querySelector('div.ReactModalPortal');
#                                 if (a) a.parentNode.removeChild(a);""")
        try:
            driver.execute_script("""var element = document.querySelector("a.next_page");
                                element.click();""")
        except:
            return

In [296]:
driver = init_driver()
get_urls(driver, URL)
driver.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [97]:
# df=pd.DataFrame({'Contact_name':Contact_name, 'Contact_Tel':Contact_Tel, 'Driftkostnad':Driftkostnad, 'TotalArea':TotalArea, 'Description':Description, 'Price':Price, 'Type':Type, 'Property_Address1':Property_Address1, 'Rooms':Rooms, 'Location':Location, 'Property_Image':Property_Image, 'Property_URL':Property_URL})

In [98]:
# df.to_csv('hemnet.se.csv')